In [ ]:
import toml
from torch.utils import data

from tools.celeba import CelebALoader
from dataset import Split

from models.models import create_model
from tools.toml import load_option
from tools.mask import mask_iter

In [ ]:
header_opt = load_option('options/header.toml')
loader = CelebALoader(header_opt.data_root)
dataset = Split(loader, header_opt.fine_size)

In [ ]:
val_data = dataset.train(['attr'], header_opt.fine_size)

In [ ]:
ty = ["attr", "identity", "bbox", "landmarks"]
val_data = dataset.train(ty)
val_data

In [ ]:
val_data[0][1]

In [ ]:
val_data = dataset.valid('bbox')
valset = data.DataLoader(val_data,
                         batch_size=header_opt.batch_size,
                         shuffle=True)
maskset = mask_iter(header_opt.mask_root, header_opt.fine_size)
print('训练集数量：', len(valset))
print('掩码数量：', len(maskset))

In [ ]:
import time
import torch
import torchvision
model_opt = load_option('options/train2.toml')

model = create_model(model_opt)

In [ ]:
load_epoch = 0
model.load(load_epoch)

In [ ]:
import time
from pathlib import Path

In [ ]:
def mask_op(mask):
    mask = mask.cuda()
    mask = mask[0][0]
    mask = torch.unsqueeze(mask, 0)
    mask = torch.unsqueeze(mask, 1)
    mask = mask.byte()
    return mask

In [ ]:
save_dir = '../measure/true'
save_dir = Path(save_dir)
if not save_dir.exists():
    save_dir.mkdir(parents=True, exist_ok=True)

epoch = 1
i = 0
total_steps = 0
for batch, mask in zip(valset, maskset):
    iter_start_time = time.time()
    image = batch[0]
    image = image.cuda()
    mask = mask_op(mask)
    model.set_input(image, mask)
    model.set_gt_latent()
    model.test()
    real_A, real_B, fake_B = model.get_current_visuals()
    total_steps += header_opt.batch_size
    pic = (torch.cat([real_A, real_B, fake_B], dim=0) + 1) / 2.0
    torchvision.utils.save_image(pic, f"{save_dir}/epoch{epoch}-{total_steps}.jpg", nrow=1)